In [18]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [14]:
def surrogate_gen():
    NUM = 70 # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate 50 points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate 50 points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [31]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 150
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fitting convex function to given data

In [19]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = 3
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [40]:
def surrogate_gen():
    
    # Set dimensions
    m = 40  # number of rows for A
    n = 80  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [20]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            #ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float)))
            ips.append((Q, q, G, h, A, b, S))
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  9%|▉         | 9/100 [00:00<00:02, 41.59it/s]

True
     pcost       dcost       gap    pres   dres
 0: -1.3867e+01 -1.3745e+01  2e+01  4e+00  2e+00
 1: -1.5076e+01 -1.6344e+01  4e+00  1e+00  6e-01
 2: -1.7210e+01 -1.7900e+01  8e-01  5e-02  3e-02
 3: -1.7213e+01 -1.7221e+01  1e-02  6e-04  3e-04
 4: -1.7213e+01 -1.7213e+01  1e-04  6e-06  3e-06
 5: -1.7213e+01 -1.7213e+01  1e-06  6e-08  3e-08
Optimal solution found.
status optimal
True
     pcost       dcost       gap    pres   dres
 0: -1.0532e+01 -1.0420e+01  2e+01  4e+00  3e+00
 1: -1.1719e+01 -1.2984e+01  4e+00  1e+00  6e-01
 2: -1.3804e+01 -1.4459e+01  7e-01  5e-02  3e-02
 3: -1.3806e+01 -1.3814e+01  9e-03  6e-04  3e-04
 4: -1.3806e+01 -1.3806e+01  9e-05  6e-06  3e-06
 5: -1.3806e+01 -1.3806e+01  9e-07  6e-08  3e-08
Optimal solution found.
status optimal
True
     pcost       dcost       gap    pres   dres
 0: -1.3152e+01 -1.2922e+01  2e+01  5e+00  2e+00
 1: -1.4587e+01 -1.5877e+01  4e+00  1e+00  6e-01
 2: -1.7224e+01 -1.8288e+01  1e+00  6e-02  3e-02
 3: -1.7232e+01 -1.7245e+01 

In [21]:
# Open and load the .pkl.gz file
pkg_idx = 1
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)

[(<6x6 matrix, tc='d'>, <6x1 matrix, tc='d'>, <9x6 matrix, tc='d'>, <9x1 matrix, tc='d'>, <0x6 matrix, tc='d'>, <0x1 matrix, tc='d'>, array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]]), 5), (<6x6 matrix, tc='d'>, <6x1 matrix, tc='d'>, <9x6 matrix, tc='d'>, <9x1 matrix, tc='d'>, <0x6 matrix, tc='d'>, <0x1 matrix, tc='d'>, array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]]), 5), (<6x6 matrix, tc='d'>, <6x1 matrix, tc='d'>, <9x6 matrix, tc='d'>, <9x1 matrix, tc='d'>, <0x6 matrix, tc='d'>, <0x1 matrix, tc='d'>, array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.,